# Introduction to Linear Regression
In this tutorial, we'll discuss one of the foundational algorithms in machine learning: Linear regression. We'll create a model that predicts crop yields for apples and oranges (target variables) by looking at the average temperature, rainfall, and humidity (input variables or features) in a region. Here's the training data:

linear-regression-training-data

In a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2
Visually, it means that the yield of apples is a linear or planar function of temperature, rainfall and humidity:

linear-regression-graph

The learning part of linear regression is to figure out a set of weights w11, w12,... w23, b1 & b2 using the training data, to make accurate predictions for new data. The learned weights will be used to predict the yields for apples and oranges in a new region using the average temperature, rainfall, and humidity for that region.

We'll train our model by adjusting the weights slightly many times to make better predictions, using an optimization technique called gradient descent. Let's begin by importing Numpy and PyTorch.

In [1]:
import numpy as np
import torch

# Training data
We can represent the training data using two matrices: inputs and targets, each with one row per observation, and one column per variable.

In [2]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [3]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [4]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


# Linear regression model from scratch
The weights and biases (w11, w12,... w23, b1 & b2) can also be represented as matrices, initialized as random values. The first row of w and the first element of b are used to predict the first target variable, i.e., yield of apples, and similarly, the second for oranges.

In [5]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[-1.0627, -0.0951, -0.1590],
        [-1.2623, -1.5845,  0.2664]], requires_grad=True)
tensor([ 1.8752, -1.7420], requires_grad=True)


In [6]:
# Our model is simply a function that performs a matrix multiplication of the inputs and the weights w (transposed) and adds the bias b (replicated for each observation

In [7]:
# We can define the model as follows:

In [8]:
def model(x):
    return x @ w.t() + b

In [9]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[ -88.9105, -188.5914],
        [-113.3751, -238.9915],
        [-112.5460, -308.4269],
        [-116.4913, -188.7677],
        [ -91.7110, -222.2992]], grad_fn=<AddBackward0>)


In [10]:
# Compare with targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


# Loss function
Before we improve our model, we need a way to evaluate how well our model is performing. We can compare the model's predictions with the actual targets using the following method:

Calculate the difference between the two matrices (preds and targets).
Square all elements of the difference matrix to remove negative values.
Calculate the average of the elements in the resulting matrix.
The result is a single number, known as the mean squared error (MSE).

In [11]:
# MSE loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [12]:
# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(71426.4219, grad_fn=<DivBackward0>)


# Compute gradients
With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t. to the weights and biases because they have requires_grad set to True. We'll see how this is useful in just a moment.

In [13]:
# Compute gradients
loss.backward()

In [14]:
# Gradients for weights
print(w)
print(w.grad)

tensor([[-1.0627, -0.0951, -0.1590],
        [-1.2623, -1.5845,  0.2664]], requires_grad=True)
tensor([[-15194.4570, -16497.7148, -10170.9561],
        [-26959.7012, -29773.7656, -18145.2012]])


# Adjust weights and biases to reduce the loss

In [ ]:
w
w.grad

In [ ]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

In [ ]:
# We use torch.no_grad to indicate to PyTorch that we shouldn't track, calculate, or modify gradients while updating the weights and biases.

In [ ]:
# Let's verify that the loss is actually lower
loss = mse(preds, targets)
print(loss)

In [19]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


# Train the model using gradient descent
As seen above, we reduce the loss and improve our model using the gradient descent optimization algorithm. Thus, we can train the model using the following steps:

Generate predictions

Calculate the loss

Compute gradients w.r.t the weights and biases

Adjust the weights by subtracting a small quantity proportional to the gradient

Reset the gradients to zero

Let's implement the above step by step.

In [20]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[ -62.3897, -141.1567],
        [ -78.5190, -176.6411],
        [ -71.3189, -234.5477],
        [ -90.1339, -141.7492],
        [ -58.2676, -162.4093]], grad_fn=<AddBackward0>)


In [21]:
# Calculate the loss
loss = mse(preds, targets)
print(loss)

tensor(48217.0547, grad_fn=<DivBackward0>)


In [22]:
# Compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[-12456.3096, -13555.1934,  -8355.2314],
        [-22061.2207, -24506.5664, -14895.7832]])
tensor([-148.3258, -263.3008])


In [23]:
# Adjust weights & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [24]:
print(w)
print(b)

tensor([[-0.7862,  0.2054,  0.0263],
        [-0.7720, -1.0417,  0.5968]], requires_grad=True)
tensor([ 1.8785, -1.7362], requires_grad=True)


In [25]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(32575.5879, grad_fn=<DivBackward0>)


# Train for multiple epochs
To reduce the loss further, we can repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an epoch. Let's train the model for 100 epochs.

In [26]:
# Train for 100 epochs
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [27]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(81.0519, grad_fn=<DivBackward0>)


In [28]:
# Predictions
preds

tensor([[ 58.4814,  73.2662],
        [ 80.8934, 106.5323],
        [119.5702, 114.8667],
        [ 26.6777,  56.2703],
        [ 96.7412, 117.7591]], grad_fn=<AddBackward0>)

In [29]:
# Targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

# Linear regression using PyTorch built-ins
We've implemented linear regression & gradient descent model using some basic tensor operations. However, since this is a common pattern in deep learning, PyTorch provides several built-in functions and classes to make it easy to create and train models with just a few lines of code.

Let's begin by importing the torch.nn package from PyTorch, which contains utility classes for building neural networks.

In [30]:
import torch.nn as nn

In [31]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [32]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

# Dataset and DataLoader
We'll create a TensorDataset, which allows access to rows from inputs and targets as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [33]:
from torch.utils.data import TensorDataset

In [34]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

#The TensorDataset allows us to access a small section of the training data using the array indexing notation ([0:3] in the above code). It returns a tuple with two elements. The first element contains the input variables for the selected rows, and the second contains the targets.

We'll also create a DataLoader, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [35]:
from torch.utils.data import DataLoader

In [36]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [37]:
# We can use the data loader in a for loop. Let's look at an example.

In [38]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 88., 134.,  59.],
        [ 73.,  66.,  44.],
        [ 68.,  97.,  70.],
        [ 69.,  96.,  70.],
        [ 91.,  88.,  64.]])
tensor([[118., 132.],
        [ 57.,  69.],
        [102., 120.],
        [103., 119.],
        [ 81., 101.]])


In each iteration, the data loader returns one batch of data with the given batch size. If shuffle is set to True, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, leading to a faster reduction in the loss.

nn.Linear
Instead of initializing the weights & biases manually, we can define the model using the nn.Linear class from PyTorch, which does it automatically.

In [39]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.0010, -0.4952,  0.5514],
        [ 0.0922,  0.4387,  0.5263]], requires_grad=True)
Parameter containing:
tensor([-0.4719,  0.2010], requires_grad=True)


In [40]:
# PyTorch models also have a helpful .parameters method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [41]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[-0.0010, -0.4952,  0.5514],
         [ 0.0922,  0.4387,  0.5263]], requires_grad=True),
 Parameter containing:
 tensor([-0.4719,  0.2010], requires_grad=True)]

In [42]:
# Generate predictions
preds = model(inputs)
preds

tensor([[-10.0126,  58.9509],
        [ -8.8498,  80.8740],
        [-34.9313,  97.5275],
        [ -1.4678,  47.9378],
        [ -9.4794,  85.5133],
        [ -9.5185,  58.6044],
        [ -7.8032,  80.9616],
        [-34.3809,  98.1460],
        [ -1.9619,  48.2843],
        [ -8.9270,  85.9474],
        [ -8.9660,  59.0385],
        [ -8.3557,  80.5275],
        [-35.9778,  97.4399],
        [ -2.0202,  47.5037],
        [ -9.9735,  85.8598]], grad_fn=<AddmmBackward>)

# Loss Function
Instead of defining a loss function manually, we can use the built-in loss function mse_loss.

In [43]:
# Import nn.functional
import torch.nn.functional as F

In [44]:
# Define loss function
loss_fn = F.mse_loss

In [45]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(5204.5562, grad_fn=<MseLossBackward>)


# Optimizer
Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer optim.SGD. SGD is short for "stochastic gradient descent". The term stochastic indicates that samples are selected in random batches instead of as a single group.

In [46]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

# Train the model
We are now ready to train the model. We'll follow the same process to implement gradient descent:

Generate predictions

Calculate the loss

Compute gradients w.r.t the weights and biases

Adjust the weights by subtracting a small quantity proportional to the gradient

Reset the gradients to zero

The only change is that we'll work batches of data instead of processing the entire training data in every iteration. Let's define a utility function fit that trains the model for a given number of epochs.

In [47]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Some things to note above:

We use the data loader defined earlier to get batches of data for every iteration.

Instead of updating parameters (weights and biases) manually, we use opt.step to perform the update and opt.zero_grad to reset the gradients to zero.

We've also added a log statement that prints the loss from the last batch of data for every 10th epoch to track training progress. loss.item returns the actual value stored in the loss tensor.

Let's train the model for 100 epochs.

In [48]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 479.1310
Epoch [20/100], Loss: 567.6870
Epoch [30/100], Loss: 137.6669
Epoch [40/100], Loss: 163.8999
Epoch [50/100], Loss: 75.5471
Epoch [60/100], Loss: 35.4632
Epoch [70/100], Loss: 62.9061
Epoch [80/100], Loss: 30.3351
Epoch [90/100], Loss: 26.5898
Epoch [100/100], Loss: 18.1267


In [49]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 58.1391,  71.2859],
        [ 84.5322,  99.6696],
        [111.0461, 133.8237],
        [ 27.9980,  42.2053],
        [101.8935, 114.3962],
        [ 57.1643,  70.2855],
        [ 84.7337,  99.5450],
        [111.5964, 134.3323],
        [ 28.9728,  43.2057],
        [103.0699, 115.2720],
        [ 58.3406,  71.1613],
        [ 83.5573,  98.6692],
        [110.8445, 133.9483],
        [ 26.8216,  41.3295],
        [102.8683, 115.3966]], grad_fn=<AddmmBackward>)

In [50]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

In [51]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[55.7292, 68.3437]], grad_fn=<AddmmBackward>)